In [ ]:
!pip install transformers
!pip install textattack[tensorflow]
!pip install torchfile

In [36]:
import torch
import textattack
from torchtext.datasets import IMDB, AG_NEWS, YahooAnswers
from torchtext.vocab import GloVe
from torchtext.data import to_map_style_dataset
from torchtext.data.utils import get_tokenizer
from torch.nn import LSTM, GRU, Linear, Softmax, CrossEntropyLoss
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, random_split, Dataset
from torch.optim import Adam
from tqdm import tqdm
import torchtext
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
class BidirectionalLSTMClassifier(torch.nn.Module):
    def __init__(self, num_classes, hidden_size, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.LSTM = LSTM(50, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.linear = Linear(2 * hidden_size, num_classes)
        self.softmax = Softmax(dim=1)
        
    def forward(self, x):
        _, (h_n, _) = self.LSTM(x)
        h_forward = h_n[2 * self.num_layers - 2]
        h_backward = h_n[2 * self.num_layers - 1]
        y = self.linear(torch.cat((h_forward, h_backward), 1))
        return self.softmax(y)

class CustomPyTorchModelWrapper(textattack.models.wrappers.model_wrapper.ModelWrapper):
    def __init__(self, model, outdim, vocab=torchtext.vocab.GloVe("6B", dim=50), tokenizer=torchtext.data.utils.get_tokenizer("basic_english")):
        self.model = model
        self.tokenizer = tokenizer
        self.outdim = outdim
        self.vocab = vocab
    
    def __call__(self, text_input_list):
        preds = torch.zeros(size=(len(text_input_list),self.outdim))
        for i, review in enumerate(text_input_list):
            tokens = self.tokenizer(review)
            input = self.vocab.get_vecs_by_tokens(tokens)
            with torch.no_grad():
                prediction = self.model(torch.unsqueeze(input,dim=0).to(device))
                preds[i] = prediction

        return preds

.vector_cache/glove.6B.zip: 862MB [02:42, 5.29MB/s]                           
100%|█████████▉| 399999/400000 [00:15<00:00, 26609.24it/s]


In [27]:
# Load Model
from pathlib import Path
from IPython import get_ipython
on_colab = 'google.colab' in str(get_ipython())

if on_colab:
  from google.colab import drive
  drive.mount("/content/gdrive")

PATH =  "/content/gdrive/My Drive/DeepLearning/MODELS/" if on_colab else "./"
PATH_RES = "/content/gdrive/My Drive/DeepLearning/RES/" if on_colab else "./"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [31]:
DATASET = 'AG_NEWS'  # choose from IMDB, AG_NEWS, YahooAnswers
MODEL = 'LSTM'  # choose from: GRU, LSTM, CNN, BERT, CNN2
num_classes = 4

In [45]:
MODEL_READ_NAME = "LSTM_AG_NEWS_WLADL"
ATTACK_NAME = "BAE" # choose from: PWWS, GA, BAE
examples = 100

In [46]:
# Load Model to be used to generate the adversarial examples for training:
model = BidirectionalLSTMClassifier(num_classes, 64, 1).to(device)
checkpoint = torch.load(PATH+MODEL_READ_NAME+".pt")
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

BidirectionalLSTMClassifier(
  (LSTM): LSTM(50, 64, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=128, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)

In [ ]:
model_wrapper = CustomPyTorchModelWrapper(model, outdim=num_classes)

if DATASET == "AG_NEWS":
    dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")
elif DATASET == "IMDB":
    dataset = textattack.datasets.HuggingFaceDataset("imdb", split="test")
elif DATASET == "YahooAnswers":
    dataset = textattack.datasets.HuggingFaceDataset("yahoo_answers", split="test")

if ATTACK_NAME == "PWWS":
    attack = textattack.attack_recipes.pwws_ren_2019.PWWSRen2019.build(model_wrapper)
elif ATTACK_NAME == "BAE":
    attack = textattack.attack_recipes.bae_garg_2019.BAEGarg2019.build(model_wrapper)
elif ATTACK_NAME == "GA":
    attack = textattack.attack_recipes.iga_wang_2019.IGAWang2019.build(model_wrapper)

attack_args = textattack.AttackArgs(num_examples=examples, log_to_csv=PATH_RES+MODEL_READ_NAME+"_"+ATTACK_NAME+".csv")
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

In [48]:
import pandas as pd
df = pd.read_csv(PATH_RES+MODEL_READ_NAME+"_"+ATTACK_NAME+".csv")
df

,ground_truth_output,num_queries,original_output,original_score,original_text,perturbed_output,perturbed_score,perturbed_text,result_type
0,2.0,86.0,2.0,2.455711e-05,Fears for T N [[pension]] after talks Unions r...,2.0,0.007293,Fears for T N [[left]] after talks Unions repr...,Failed
1,3.0,237.0,3.0,3.099442e-06,The Race is On: Second Private Team Sets [[Lau...,3.0,0.000039,The Race is On: Second Private Team Sets [[rec...,Failed
2,3.0,85.0,3.0,5.614758e-05,Ky. Company Wins Grant to Study [[Peptides]] (...,3.0,0.000336,Ky. Company Wins Grant to Study [[peptide]] (A...,Failed
3,3.0,156.0,3.0,8.217931e-03,Prediction Unit Helps Forecast Wildfires (AP) ...,3.0,0.498610,Prediction Unit Helps Forecast Wildfires (AP) ...,Failed
4,3.0,89.0,3.0,1.972616e-03,Calif. [[Aims]] to Limit Farm-Related Smog (AP...,0.0,0.701632,Calif. [[campaign]] to Limit Farm-Related Smog...,Successful
...,...,...,...,...,...,...,...,...,...
95,1.0,146.0,1.0,9.536743e-07,Eriksson doesn #39;t feel any extra pressure f...,1.0,0.000004,Eriksson doesn #39;t feel any extra pressure f...,Failed
96,1.0,102.0,1.0,4.291534e-06,Injured [[Heskey]] to miss England friendly NE...,1.0,0.000027,Injured [[plans]] to miss England friendly NEW...,Failed
97,2.0,315.0,2.0,7.152557e-07,"Staples [[Profit]] Up, to Enter China Market ...",2.0,0.000015,"Staples [[team]] Up, to Enter China Market NE...",Failed
98,0.0,100.0,0.0,3.933907e-06,Delegation Is Delayed Before Reaching [[Najaf]...,0.0,0.000015,Delegation Is Delayed Before Reaching [[lake]]...,Failed
